# Data Pre Processing

This is a notebook for intaking a raw data file, preprocessing it, and exporting it in a format in which it can easily be labeled.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/directoryident.tsv", sep="\t")
df.head()

,dn,lastname,givenname,middle,dob,address
0,"uid=madanafl,ou=People,dc=umich,dc=edu",Madanay,Farrah,L,1990091200Z,NaN
1,"uid=jocbran,ou=People,dc=umich,dc=edu",Branham,Jocelyn,NaN,20050616000000Z,{addr1=345A Infantry Ter}:{city=San Francisco}...
2,"uid=stoicabr,ou=People,dc=umich,dc=edu",Stoica,Brooke,NaN,20031007000000Z,{addr1=9051 Salem Farms Drive}:{city=South Lyo...
3,"uid=heavens,ou=People,dc=umich,dc=edu",Smith,Heaven,NaN,19880104000000Z,{addr1=6655 Jackson Rd Unit 191}:{city=Ann Arb...
4,"uid=funnim,ou=People,dc=umich,dc=edu",Funni,Matthew,NaN,1989121900Z,{addr1=2095 pauls way}:{city=Commerce twp}:{st...


## Preprocessing

### Cleanup

In [3]:
# Add unique ID column
# Uses the row number of original csv file as unique ID
df['Unique ID'] = df.index + 1

In [4]:
# Turn all NaN values into empty strings
df['address'] = df['address'].fillna(" ")
df['middle'] = df['middle'].fillna(" ")

In [5]:
df.head()

,dn,lastname,givenname,middle,dob,address,Unique ID
0,"uid=madanafl,ou=People,dc=umich,dc=edu",Madanay,Farrah,L,1990091200Z,,1
1,"uid=jocbran,ou=People,dc=umich,dc=edu",Branham,Jocelyn,,20050616000000Z,{addr1=345A Infantry Ter}:{city=San Francisco}...,2
2,"uid=stoicabr,ou=People,dc=umich,dc=edu",Stoica,Brooke,,20031007000000Z,{addr1=9051 Salem Farms Drive}:{city=South Lyo...,3
3,"uid=heavens,ou=People,dc=umich,dc=edu",Smith,Heaven,,19880104000000Z,{addr1=6655 Jackson Rd Unit 191}:{city=Ann Arb...,4
4,"uid=funnim,ou=People,dc=umich,dc=edu",Funni,Matthew,,1989121900Z,{addr1=2095 pauls way}:{city=Commerce twp}:{st...,5


In [6]:
# Convert all columns to string
df = df.astype(str)

In [7]:
# take sample of 5000
df = df.sample(n=500)

In [8]:
# Function to parse a singular dn field
# Returns the uid in the dn
def parse_dn(dn):
    dn = dn.split(',')
    dn = [x.split('=') for x in dn]
    dn = {x[0].strip(): x[1].strip() for x in dn}
    return dn["uid"]

In [9]:
# Function to concatenate every value in the address 
# We want to concatenate all values inside curly braces
def concat_address(address):
    if address != ' ':
        address = address.split('}:{')
        try: 
            address = [x.split('=')[1] for x in address]
        except:
            return "DELETE"
        address = ' '.join(address)
    return address

In [10]:
# Use vectorization to set new values for dn and address columns
df['dn'] = df['dn'].apply(parse_dn)
df['address'] = df['address'].apply(concat_address)
df['dob'] = df['dob'].apply(lambda x : x[:8])

In [11]:
if "DELETE" in df['address'].values:
    df = df[df['address'] != "DELETE"]

df.head()

,dn,lastname,givenname,middle,dob,address,Unique ID
528530,mfs,Simonte,Maria,Fomin,19660831,22526 Autumn Park Blvd Novi MI 48374-3639 UNIT...,528531
46580,suriang,Chaniang,Surachai,,19830330,400 North Ingalls Street 48109 USA Ann Arbor MI},46581
359294,keweegal,Sherk,Nancy,Pairolero,19390511,702 Clark Xing SE Grand Rapids MI 49506-3300 U...,359295
510142,corlett,Corlett,Michael,Sean,19710617,4121 Arkansas Ave NW Washington DC 20011-5505 ...,510143
11627,xingxian,Li,Laurel,,20030616,415 South Street Waltham MA 02453 UNITED STATE...,11628


In [12]:
# Change column names
df.columns = ['ID', 'Last Name', 'First Name', 'Middle Name', 'Date of Birth', 'Address', 'Unique ID']
# Reorder columns
df = df[['Unique ID', 'ID', 'First Name', 'Middle Name', 'Last Name', 'Date of Birth', 'Address']]

In [13]:
df.head()

,Unique ID,ID,First Name,Middle Name,Last Name,Date of Birth,Address
528530,528531,mfs,Maria,Fomin,Simonte,19660831,22526 Autumn Park Blvd Novi MI 48374-3639 UNIT...
46580,46581,suriang,Surachai,,Chaniang,19830330,400 North Ingalls Street 48109 USA Ann Arbor MI}
359294,359295,keweegal,Nancy,Pairolero,Sherk,19390511,702 Clark Xing SE Grand Rapids MI 49506-3300 U...
510142,510143,corlett,Michael,Sean,Corlett,19710617,4121 Arkansas Ave NW Washington DC 20011-5505 ...
11627,11628,xingxian,Laurel,,Li,20030616,415 South Street Waltham MA 02453 UNITED STATE...


### Similarity Measures

In [14]:
# Sample 50 records from df into new df
df_sample = df.sample(50)

In [15]:
df_sample.head()

,Unique ID,ID,First Name,Middle Name,Last Name,Date of Birth,Address
18362,18363,likatie,Katie,,Liu,19971111,3430 Nixon Rd Unit 213 Ann Arbor MI 48105 UNIT...
546303,546304,dixongre,Gregory,Thomas,Dixon,19540317,16602 Burke Ln Huntington Beach CA 92647-4536 ...
162,163,rselven,Ranya,,Selven,20061219,1924 Bloomfield Oaks Drive 48324 USA West Bloo...
68482,68483,whitmas,Mason,,Whitney,20021211,
446214,446215,weverard,W Curtis,,Everard,19441220,


In [16]:
# Remove sampled records from df
df = df.drop(df_sample.index)

With a sample df, we can now build a similarity measure df by comparing each record in the sample df with the other records.

In [17]:
import Levenshtein as lev

In [18]:
"""
Returns normalized levenshtein distance between two strings
"""
def levenshtein_distance(
        s1: str,
        s2: str
) -> int:
    distance = lev.distance(s1, s2)
    try:
        return 1 - distance / float(max(len(s1), len(s2)))
    except ZeroDivisionError:
        return 0

In [19]:
"""
Returns Levenshtein distance between each field of two rows
Assumes following format for rows:
    row = pd.Series([First Name, Middle Name, Last Name, DOB, Address, ID])
"""
def row_similarity(
        row_1: pd.Series, 
        df_2: pd.DataFrame
    ) -> pd.DataFrame:

    # Compute similarity measures for each column using Levenshtein distance
    first_name_similarity = df_2['First Name'].apply(lambda x : levenshtein_distance(row_1['First Name'], x))
    middle_name_similarity = df_2['Middle Name'].apply(lambda x : levenshtein_distance(row_1['Middle Name'], x))
    last_name_similarity = df_2['Last Name'].apply(lambda x : levenshtein_distance(row_1['Last Name'], x))
    dob_similarity = df_2['Date of Birth'].apply(lambda x : levenshtein_distance(row_1['Date of Birth'], x))
    address_similarity = df_2['Address'].apply(lambda x : levenshtein_distance(row_1['Address'], x))
    id_similarity = df_2['ID'].apply(lambda x : levenshtein_distance(row_1['ID'], x))

    # Return DataFrame with similarity measures
    return pd.DataFrame({
        'UNIQ_ID1': row_1['Unique ID'],
        'UNIQ_ID2': df_2['Unique ID'],
        'ID1': row_1['ID'],
        'ID2': df_2['ID'],
        'ID Similarity': id_similarity,
        'First Name 1' : row_1['First Name'],
        'First Name 2' : df_2['First Name'],
        'First Name Similarity': first_name_similarity,
        'Middle Name 1' : row_1['Middle Name'],
        'Middle Name 2' : df_2['Middle Name'],
        'Middle Name Similarity': middle_name_similarity,
        'Last Name 1' : row_1['Last Name'],
        'Last Name 2' : df_2['Last Name'],
        'Last Name Similarity': last_name_similarity,
        'Date of Birth 1' : row_1['Date of Birth'],
        'Date of Birth 2' : df_2['Date of Birth'],
        'Date of Birth Similarity': dob_similarity,
        'Address 1' : row_1['Address'],
        'Address 2' : df_2['Address'],
        'Address Similarity': address_similarity
    })


In [20]:
"""
Builds similarity measure between records in two df
Creates a new df from the two df with the following columns:
    - ID1: ID of record in df1
    - ID2: ID of record in df2
    - First Name Similarity: Normalized levenshtein distance between first names
    - Middle Name Similarity: Normalized levenshtein distance between middle names
    - Last Name Similarity: Normalized levenshtein distance between last names
    - Date of Birth Similarity: Normalized levenshtein distance between dates of birth
    - Address Similarity: Normalized levenshtein distance between addresses
    - ID Similarity: Normalized levenshtein distance between IDs
"""
def build_similarity_df (
        df_1: pd.DataFrame,
        df_2: pd.DataFrame
) -> pd.DataFrame:
    new_df = pd.DataFrame(columns=[ 'UNIQ_ID1',
                                    'UNIQ_ID2',
                                    'ID1', 
                                   'ID2', 
                                   'ID Similarity',
                                   'First Name 1',
                                   'First Name 2',
                                   'First Name Similarity', 
                                   'Middle Name 1',
                                    'Middle Name 2',
                                   'Middle Name Similarity',
                                    'Last Name 1',
                                    'Last Name 2', 
                                   'Last Name Similarity', 
                                    'Date of Birth 1',
                                    'Date of Birth 2',
                                   'Date of Birth Similarity', 
                                    'Address 1',
                                    'Address 2',
                                   'Address Similarity'])

    # Convert ID columns to string
    df_1["ID"] = df_1["ID"].astype(str)
    df_2["ID"] = df_2["ID"].astype(str)

    def apply_row_similarity(row, new_df):
        sim = row_similarity(row, df_2)
        new_df = pd.concat([new_df, sim], ignore_index=True)
        return new_df

    # Use vectorization to compute similarity between each row in df_1 and df_2
    new_df = df_1.apply(apply_row_similarity, args=(new_df,), axis=1).reset_index(drop=True)

    # New df is a series of dfs, so we need to concatenate them
    new_df = pd.concat(new_df.to_list(), ignore_index=True)

    return new_df

In [21]:
similarity_df = build_similarity_df(df_sample,  df)
similarity_df.head()

,UNIQ_ID1,UNIQ_ID2,ID1,ID2,ID Similarity,First Name 1,First Name 2,First Name Similarity,Middle Name 1,Middle Name 2,Middle Name Similarity,Last Name 1,Last Name 2,Last Name Similarity,Date of Birth 1,Date of Birth 2,Date of Birth Similarity,Address 1,Address 2,Address Similarity
0,18363,528531,likatie,mfs,0.00,Katie,Maria,0.400000,,Fomin,0.0,Liu,Simonte,0.142857,19971111,19660831,0.375,3430 Nixon Rd Unit 213 Ann Arbor MI 48105 UNIT...,22526 Autumn Park Blvd Novi MI 48374-3639 UNIT...,0.383333
1,18363,46581,likatie,suriang,0.00,Katie,Surachai,0.125000,,,1.0,Liu,Chaniang,0.125000,19971111,19830330,0.250,3430 Nixon Rd Unit 213 Ann Arbor MI 48105 UNIT...,400 North Ingalls Street 48109 USA Ann Arbor MI},0.300000
2,18363,359295,likatie,keweegal,0.00,Katie,Nancy,0.200000,,Pairolero,0.0,Liu,Sherk,0.000000,19971111,19390511,0.500,3430 Nixon Rd Unit 213 Ann Arbor MI 48105 UNIT...,702 Clark Xing SE Grand Rapids MI 49506-3300 U...,0.428571
3,18363,510143,likatie,corlett,0.00,Katie,Michael,0.142857,,Sean,0.0,Liu,Corlett,0.000000,19971111,19710617,0.500,3430 Nixon Rd Unit 213 Ann Arbor MI 48105 UNIT...,4121 Arkansas Ave NW Washington DC 20011-5505 ...,0.437500
4,18363,11628,likatie,xingxian,0.25,Katie,Laurel,0.333333,,,1.0,Liu,Li,0.666667,19971111,20030616,0.125,3430 Nixon Rd Unit 213 Ann Arbor MI 48105 UNIT...,415 South Street Waltham MA 02453 UNITED STATE...,0.466667


### Discretization

### WPKID

In [54]:
wpkid_df = similarity_df.copy()

In [55]:
# holds bin ranges for each similarity attribute
bin_ranges = {'First Name Similarity' : [],
        'Middle Name Similarity' : [],
        'Last Name Similarity' : [],
        'Date of Birth Similarity' : [],
        'Address Similarity' : [],
        'ID Similarity' : []}

In [56]:
#for solving systems of equations
import sympy as sp 
import math


t = # of intervals<br>
s = interval size<br>
n = # of instances in training dataset<br>

s * t = n<br>
s - 30 = t<br>


In [57]:
# WPKID function to solve for s and t given n, returns pair {s, t}
def interval_size_and_num(n) -> tuple:
    s, t = sp.symbols('s t')
    equation1 = sp.Eq(s * t, n)
    equation2 = sp.Eq(s - 30, t)  
    solution = sp.solve((equation1, equation2), (s, t))
    s_val = solution[0][1] * -1
    t_val = solution[1][1]

    s_int = math.floor(s_val)
    t_int = math.floor(t_val)
    return s_int, t_int

In [58]:
# function that takes in a column and discretizes it, saving bin ranges
def discretize_column(column, df):
    # separate zeros from nonzeros
    zeros_df = wpkid_df[wpkid_df[column] == 0].copy()
    nonzeros_df = wpkid_df[wpkid_df[column] != 0].copy()

    #get num_instances and num_intervals for nonzeros
    num_instances = interval_size_and_num(len(nonzeros_df))[0]
    num_intervals = interval_size_and_num(len(nonzeros_df))[1]

    # get bin ranges for nonzeros
    nonzeros_df[column], bin = pd.qcut(nonzeros_df[column], 
                                        q=num_intervals, 
                                        labels=False,
                                        retbins=True,
                                        duplicates='drop')
    
    #add 1 to every value to switch to index 1
    nonzeros_df[column] = nonzeros_df[column] + 1

    #save bin ranges
    bin_ranges[column].extend(bin.tolist())

    #recombine zero and nonzero dataframes
    df.loc[df[column] == 0, column] = zeros_df[column]
    df.loc[df[column] != 0, column] = nonzeros_df[column] 

    return df

In [59]:
wpkid_df = discretize_column('First Name Similarity', wpkid_df)
wpkid_df = discretize_column('Middle Name Similarity', wpkid_df)
wpkid_df = discretize_column('Last Name Similarity', wpkid_df)
wpkid_df = discretize_column('Date of Birth Similarity', wpkid_df)
wpkid_df = discretize_column('Address Similarity', wpkid_df)
wpkid_df = discretize_column('ID Similarity', wpkid_df)

In [60]:
# insert 0 at the beginning of each bin range
for key in bin_ranges:
    bin_ranges[key].insert(0, 0)

# if last value in bin range is not 1.0, add it
for key in bin_ranges:
    if bin_ranges[key][-1] != 1.0:
        bin_ranges[key].append(1.0)

In [64]:
wpkid_df.head()

,UNIQ_ID1,UNIQ_ID2,ID1,ID2,ID Similarity,First Name 1,First Name 2,First Name Similarity,Middle Name 1,Middle Name 2,Middle Name Similarity,Last Name 1,Last Name 2,Last Name Similarity,Date of Birth 1,Date of Birth 2,Date of Birth Similarity,Address 1,Address 2,Address Similarity
0,18363,528531,likatie,mfs,0.0,Katie,Maria,15.0,,Fomin,0.0,Liu,Simonte,8.0,19971111,19660831,2.0,3430 Nixon Rd Unit 213 Ann Arbor MI 48105 UNIT...,22526 Autumn Park Blvd Novi MI 48374-3639 UNIT...,44.0
1,18363,46581,likatie,suriang,0.0,Katie,Surachai,4.0,,,14.0,Liu,Chaniang,7.0,19971111,19830330,1.0,3430 Nixon Rd Unit 213 Ann Arbor MI 48105 UNIT...,400 North Ingalls Street 48109 USA Ann Arbor MI},41.0
2,18363,359295,likatie,keweegal,0.0,Katie,Nancy,8.0,,Pairolero,0.0,Liu,Sherk,0.0,19971111,19390511,3.0,3430 Nixon Rd Unit 213 Ann Arbor MI 48105 UNIT...,702 Clark Xing SE Grand Rapids MI 49506-3300 U...,57.0
3,18363,510143,likatie,corlett,0.0,Katie,Michael,5.0,,Sean,0.0,Liu,Corlett,0.0,19971111,19710617,3.0,3430 Nixon Rd Unit 213 Ann Arbor MI 48105 UNIT...,4121 Arkansas Ave NW Washington DC 20011-5505 ...,62.0
4,18363,11628,likatie,xingxian,3.0,Katie,Laurel,13.0,,,14.0,Liu,Li,23.0,19971111,20030616,1.0,3430 Nixon Rd Unit 213 Ann Arbor MI 48105 UNIT...,415 South Street Waltham MA 02453 UNITED STATE...,81.0


In [62]:
for key, value in bin_ranges.items():
    print(key, value)

First Name Similarity [0, 0.05882352941176472, 0.0714285714285714, 0.09999999999999998, 0.11111111111111116, 0.125, 0.1428571428571429, 0.16666666666666663, 0.17110617832037736, 0.19999999999999996, 0.2222222222222222, 0.25, 0.2857142857142857, 0.30000000000000004, 0.33333333333333337, 0.375, 0.4, 0.4285714285714286, 0.5, 0.6, 1.0]
Middle Name Similarity [0, 0.06666666666666665, 0.09090909090909094, 0.11111111111111116, 0.125, 0.1428571428571429, 0.16666666666666663, 0.19999999999999996, 0.2222222222222222, 0.25, 0.2857142857142857, 0.33333333333333337, 0.4, 0.4285714285714286, 0.5, 1.0]
Last Name Similarity [0, 0.050000000000000044, 0.0625, 0.07692307692307687, 0.08333333333333337, 0.09090909090909094, 0.09999999999999998, 0.11111111111111116, 0.125, 0.1428571428571429, 0.15384615384615385, 0.16666666666666663, 0.18181818181818177, 0.1875, 0.19999999999999996, 0.2222222222222222, 0.25, 0.2857142857142857, 0.30000000000000004, 0.33333333333333337, 0.375, 0.4, 0.4285714285714286, 0.5, 1

In [63]:
# export wpkid_df to csv 
wpkid_df.to_csv('data/sampleunlabeledident.csv', index=False)